In [ ]:
!pip install pycuda
!pip install gputil
!pip install nvidia-ml-py3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!nvidia-smi

Mon Jun 27 04:56:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    30W /  70W |  15106MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
import time
import GPUtil
import nvidia_smi
from pycuda.compiler import SourceModule

mod = SourceModule(""" 
        __global__ void multiplyMatrixOneBlock(float *a,float *b, float *c, int colsA,int colsB){
              int idxC = threadIdx.x+threadIdx.y*colsB;
              int idxA = threadIdx.y*colsA;
              int idxB = threadIdx.x;

              int i=0;

              for(i;i<colsA;i++){
                c[idxC] += a[idxA+i]*b[idxB];
                idxB += colsB;
              }

       
        }
        """)

In [ ]:
import numpy as np

#lista_memorias =[]
#lista_tiempos =[]

for i in range(0,10000):
  
  a = np.random.randn(20,10).astype(dtype=np.float32)
  b = np.random.randn(10,10).astype(dtype=np.float32)
  # print(b)
  res = a.dot(b)

  c = np.zeros_like(res)
  # print(c)

  a_gpu = cuda.mem_alloc(a.nbytes)
  cuda.memcpy_htod(a_gpu,a)

  b_gpu = cuda.mem_alloc(b.nbytes)
  cuda.memcpy_htod(b_gpu,b)

  c_gpu = cuda.mem_alloc(c.nbytes)
  cuda.memcpy_htod(c_gpu,c)

  func = mod.get_function("multiplyMatrixOneBlock")
  inicio = time.time()
  func(a_gpu,b_gpu,c_gpu,np.int32(a.shape[1]),np.int32(b.shape[1]),block=(np.int(b.shape[1]),np.int(a.shape[0]),1),grid=(1,1,1))

  cuda.memcpy_dtoh(c,c_gpu)

  fin = time.time()
  total = fin-inicio
  #lista_tiempos.append(total)
  #####
  nvidia_smi.nvmlInit()

  handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
  info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
  #lista_memorias.append(info.used)

  nvidia_smi.nvmlShutdown()

#print(res)
#print("\n")
#print(c)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(""" 
        __global__ void multiplyMatrixMultipleBlocksaaa(float *a,float *b, float *c,int colsB,int rowsA,int colsA){
              long idxC = threadIdx.x + blockDim.x*blockIdx.x + threadIdx.y*colsB + blockDim.y*blockIdx.y*colsB;
              long idxA = (threadIdx.y + blockDim.y*blockIdx.y)*colsA;
              long idxB = blockDim.x*blockIdx.x+threadIdx.x;

              if(idxB > colsB) return;
              if(threadIdx.y + blockDim.y*blockIdx.y > rowsA) return;
             

              for(int i = 0;i<colsA;i++){
                c[idxC] += a[idxA+i]*b[idxB];
                idxB += colsB;
              }
            }
        """)

In [ ]:
import numpy as np
import math

a = np.random.randn(90,64).astype(dtype=np.float32)
b = np.random.randn(64,50).astype(dtype=np.float32)

res = a.dot(b)

c = np.zeros_like(res)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu,a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu,b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("multiplyMatrixMultipleBlocksaaa")
func(a_gpu, b_gpu, c_gpu,np.int32(b.shape[1]),np.int32(a.shape[0]),np.int32(a.shape[1]),block=(32, 32, 1), grid=(math.ceil(b.shape[1]/32), math.ceil(a.shape[0]/32), 1))

cuda.memcpy_dtoh(c,c_gpu)
print(res)
print("\n")
print(c)

[[ -1.3059633   5.2727165 -23.441368  ...  -1.7628243  12.510549
   -8.608812 ]
 [-15.746309   10.035585    8.553311  ...  -0.627479    9.389496
   14.177984 ]
 [  5.8707113  -8.202614   -7.762378  ... -12.670758    2.3975053
   -1.2576631]
 ...
 [  6.1570163   1.2262033   2.6823974 ...  10.269517    9.612096
   -1.6647763]
 [  8.11547    -4.360548   -3.1891556 ...  -8.371554   -9.372368
   13.068209 ]
 [ -7.5932937 -14.649884    5.468161  ...   1.6329482  10.795061
  -15.9246855]]


[[ -1.305964    5.272716  -23.44137   ...  -1.7628243  12.510549
   -8.608812 ]
 [-15.74631    10.035585    8.553311  ...  -0.627479    9.389496
   14.177984 ]
 [  5.870712   -8.202614   -7.762378  ... -12.670758    2.3975053
   -1.2576631]
 ...
 [  6.1570163   1.2262033   2.6823974 ...  10.269515    9.612095
   -1.6647763]
 [  8.11547    -4.360548   -3.1891556 ...  -8.371554   -9.372368
   13.068209 ]
 [ -7.5932937 -14.649884    5.468161  ...   1.6329482  10.795061
  -15.9246855]]


In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(""" 
        __global__ void multiplyMatrixMultipleBlocksSharedMemory(float *a,float *b, float *c,int rowsA, int colsA,
                                                                  int rowsB,int colsB,int rowsC,int colsC){
              float cValue = 0;

              int row = blockIdx.y * blockDim.y + threadIdx.y;
              int col = blockIdx.x * blockDim.y + threadIdx.x;

              __shared__ float aVal[32][32];
              __shared__ float bVal[32][32];

              for(int i = 0; i < (blockDim.y + colsA - 1)/blockDim.y;i++){
                if(i*blockDim.y + threadIdx.x < colsA && row < rowsA){
                  aVal[threadIdx.y][threadIdx.x] = a[row*colsA + i*blockDim.y + threadIdx.x];
                } else {
                  aVal[threadIdx.y][threadIdx.x] = 0.0;
                }

                if(i * blockDim.y + threadIdx.y < rowsB && col < colsB){
                  bVal[threadIdx.y][threadIdx.x] = b[(i*blockDim.y + threadIdx.y)* colsB + col];
                } else {
                  bVal[threadIdx.y][threadIdx.x] = 0.0;
                }

                __syncthreads();

                for(int j = 0 ; j < blockDim.y ; ++j){
                  cValue += aVal[threadIdx.y][j] * bVal[j][threadIdx.x];
                }

                __syncthreads();

                if(row < rowsC && col < colsC){
                  c[((blockIdx.y * blockDim.y) + threadIdx.y) * colsC + (blockIdx.x * blockDim.x) + threadIdx.x] = cValue;
                }                
              } 
            }
        """)

In [ ]:
import numpy as np
import math

a = np.random.randn(40,64).astype(dtype=np.float32)
b = np.random.randn(64,90).astype(dtype=np.float32)

res = a.dot(b)

c = np.zeros_like(res)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu,a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu,b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("multiplyMatrixMultipleBlocksSharedMemory")
func(a_gpu, b_gpu, c_gpu,np.int32(a.shape[0]),np.int32(a.shape[1]),np.int32(b.shape[0]),np.int32(b.shape[1]),
     np.int32(a.shape[0]),np.int32(b.shape[1]),block=(32, 32, 1), grid=(math.ceil(b.shape[1]/32), math.ceil(a.shape[0]/32), 1))

cuda.memcpy_dtoh(c,c_gpu)
print(res)
print("\n")
print(c)

[[ -6.120091     1.2005147    0.7178729  ...   1.5885894    4.0553265
    4.7310505 ]
 [ 18.69796      0.7628877   22.032364   ...  10.950594   -10.233827
   -2.420096  ]
 [ -0.95253897  -1.4175084  -13.427255   ...  -6.356392     2.422937
    5.0694537 ]
 ...
 [  4.5174427    2.5085945   17.180086   ...  -0.03370953  -1.565575
   -7.4512305 ]
 [-13.457348    21.995415     6.188098   ... -14.525131    -7.1377077
   -4.1719966 ]
 [ 17.355614    -0.93318725  -3.6269932  ...  12.799575    -6.1824155
    0.9810536 ]]


[[ -6.120091     1.2005137    0.7178738  ...   1.5885894    4.0553265
    4.7310505 ]
 [ 18.69796      0.76288825  22.032368   ...  10.950594   -10.233827
   -2.420096  ]
 [ -0.95253885  -1.4175075  -13.427256   ...  -6.356392     2.422937
    5.0694537 ]
 ...
 [  4.517442     2.5085943   17.180084   ...  -0.03370976  -1.565575
   -7.4512305 ]
 [-13.457349    21.995415     6.1880965  ... -14.525131    -7.1377077
   -4.1719966 ]
 [ 17.355616    -0.9331864   -3.6269932  ...  1

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(""" 

        __global__ void multiplyMatrixMultipleBlocksSharedMemory(float *a,float *b, float *c,int rowsA, int colsA,
                                                                  int rowsB,int colsB,int rowsC,int colsC){
              float cValue = 0;

              int row = blockIdx.y * blockDim.y + threadIdx.y;
              int col = blockIdx.x * blockDim.y + threadIdx.x;

              __shared__ float aVal[32][32];
              __shared__ float bVal[32][32];

              for(int i = 0; i < (colsA - 1)/blockDim.x + 1;i++){
                if(i*blockDim.y + threadIdx.x < colsA && row < rowsA){
                  aVal[threadIdx.y][threadIdx.x] = a[row*colsA + i*blockDim.x + threadIdx.x];
                } else {
                  aVal[threadIdx.y][threadIdx.x] = 0.0;
                }

                if(i * blockDim.x + threadIdx.y < colsB && col < rowsB){
                  bVal[threadIdx.y][threadIdx.x] = b[(i*blockDim.y + threadIdx.y)+ colsB * col];
                } else {
                  bVal[threadIdx.y][threadIdx.x] = 0.0;
                }

                __syncthreads();

                for(int j = 0 ; j < blockDim.y ; ++j){
                  cValue += aVal[threadIdx.y][j] * bVal[j][threadIdx.x];
                }

                __syncthreads();

                if(row < rowsC && col < colsC){
                  c[row*colsC+col] = cValue;
                }                 
              } 
            }
        """)

In [ ]:
import numpy as np
import math

lista_memorias =[]
lista_tiempos =[]


for i in range(0,1):

  a = np.random.randn(100,200).astype(dtype=np.float32)
  b = np.random.randn(200,200).astype(dtype=np.float32)

  a_rows = np.int32(a.shape[0])
  a_cols = np.int32(a.shape[1])

  b_rows = np.int32(b.shape[0])
  b_cols = np.int32(b.shape[1])

  res = a.dot(np.transpose(b))

  c = np.zeros_like(res)

  a_gpu = cuda.mem_alloc(a.nbytes)
  cuda.memcpy_htod(a_gpu,a)

  b_gpu = cuda.mem_alloc(b.nbytes)
  cuda.memcpy_htod(b_gpu,b)

  c_gpu = cuda.mem_alloc(c.nbytes)
  cuda.memcpy_htod(c_gpu,c)

  func = mod.get_function("multiplyMatrixMultipleBlocksSharedMemory")
  inicio = time.time()
  func(a_gpu, b_gpu, c_gpu,a_rows,a_cols,b_rows,b_cols,
      a_rows,b_rows,block=(32, 32, 1), grid=(math.ceil(b_rows/32), math.ceil(a_rows/32), 1))

  cuda.memcpy_dtoh(c,c_gpu)


  fin = time.time()
  total = fin-inicio
  lista_tiempos.append(total)
  #####
  nvidia_smi.nvmlInit()

  handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
  info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
  lista_memorias.append(info.used)

  nvidia_smi.nvmlShutdown()

  a_gpu.free()
  b_gpu.free()
  c_gpu.free()


#print(res)
#print("\n")
#print(c)

MemoryError: ignored

In [ ]:

# importing pandas as pd  
import pandas as pd  
  
# list of name, degree, score 
     
# dictionary of lists  
dict = {'Tiempos': lista_tiempos}  
       
df = pd.DataFrame(dict) 
    
# saving the dataframe 
df.to_csv('GAA3.csv') 